In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "20"
os.environ["MKL_NUM_THREADS"] = "20"
os.environ["NUMEXPR_NUM_THREADS"] = "20"
from importlib import reload
import yee_uchie_QM_pml_class
reload(yee_uchie_QM_pml_class)
from yee_uchie_QM_pml_class import Source, Recorder, QM_UCHIE_params, Yee_UCHIE
import QM_update as QM
import numpy as np
import scipy.constants as ct
import time
from scipy.special import hankel2
import PyQt5
import matplotlib
matplotlib.use('Qt5Agg')  # Make sure to set the backend before importing pyplot

import matplotlib.pyplot as plt
%matplotlib qt

 # Make sure to set the backend before importing pyplot





In [2]:
eps0 = ct.epsilon_0
mu0 = ct.mu_0
hbar = ct.hbar #J⋅s
m = ct.electron_mass*0.15
q = -ct.elementary_charge
c0 = ct.speed_of_light 


Z0 = np.sqrt(mu0/eps0)


In [3]:

########## Fill in the parameters here ################
Nx = 400
Ny = 400
Nt = 100000

dx = 0.25e-10 # m
dy = 0.25e-10 # ms
courant = 0.9 # !Courant number, for stability this should be smaller than 1
dt = courant * 1/(np.sqrt(1/dx**2 + 1/dy**2)*ct.c)

Ly = 3/5*Ny*dy
n = 5 #@ numbers of Subgridding in one grid
N_sub = 15 #@ How much grid you want to take to subgrid

x_sub1 = Nx//3*dx #@ The locationwhere the first subgridding should happen
x_sub2 = 2*Nx//3*dx #@ The locationwhere the first subgridding should happen

QMxpos1 = n*N_sub//2
QMxpos2 = n*N_sub//2


NyQM = int(2*Ny/5)

#create the source
xs = 1/4*Nx*dx
ys = 3*Ny/5*dy
tc = dt*Nt/30
sigma = tc/6
J0 = 1e2/dx/dy
source = Source(xs, ys, J0, tc, sigma)

N = 1e24#particles/m2
#NyQM = int(2*Ny/3)
order = 'fourth'
omega = 50e14 #[rad/s]
alpha = 0

pml_nl = 20
pml_m = 4



In [4]:
potential = QM.Potential(m,omega, NyQM, dy)
#potential.V()

QMscheme1 = QM.QM(order,NyQM,dy, dt, hbar, m, q, alpha, potential, omega, N)
QMscheme2 = QM.QM(order,NyQM,dy, dt, hbar, m, q, alpha, potential, omega, N)

recorder1 = Recorder(0.74*Nx*dx, 0.5*Ny*dy)
recorder2 = Recorder(0.40*Nx*dx, 0.5*Ny*dy)
recorders = [recorder1, recorder2]

params = [QM_UCHIE_params(Ly, n, N_sub, x_sub1, QMxpos1, QMscheme1), QM_UCHIE_params(Ly, n, N_sub, x_sub2, QMxpos2, QMscheme2 )]
#params = [QM_UCHIE_params(Ly, n, N_sub, x_sub1, QMxpos1, QMscheme1)]


start_time = time.time()
test = Yee_UCHIE(Nx, Ny, Nt, dx, dy, dt, [source], pml_nl, pml_m, qm_uchie_params = params, recorders=recorders, coupled = True)
test.calculate_fields()
end_time = time.time()
print("Execution time:", end_time - start_time, "seconds")



  0%|          | 464/100000 [00:01<06:43, 246.94it/s]

In [ ]:
%matplotlib qt
fig, ax = plt.subplots()

ax.set_xlabel('Time [s]')
ax.set_ylabel('$H_{z}$ [A/m]')

# Plot the main data
ax.plot(recorder1.data_time, np.array(recorder2.data)/mu0, label = 'recorder after 1st subgridding')
ax.plot(recorder2.data_time, np.array(recorder1.data)/mu0, label = 'recorder after 2nd subgridding')
ax.legend()
plt.show()


In [ ]:
%matplotlib qt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.ticker as ticker

fig, ax = plt.subplots()

ax.set_xlabel('Time [s]')
ax.set_ylabel('$H_{z}$ [A/m]')

# Plot the main data
ax.plot(recorder1.data_time, np.array(recorder2.data)/mu0, label = 'recorder after 1st subgridding')
ax.plot(recorder2.data_time, np.array(recorder1.data)/mu0, label = 'recorder after 2nd subgridding')
ax.legend()

# Create an inset axes object with a size of 30% of the parent axes in both dimensions
# Adjust the bbox_to_anchor values to position the inset a bit higher than the bottom right corner
axins = inset_axes(ax, width="69.05%", height="30%", loc=3, bbox_to_anchor=(0.209, 0.1, 1, 1), bbox_transform=ax.transAxes)

# Plot the same data in the inset, but with different x and y limits
axins.plot(recorder1.data_time, np.array(recorder2.data)/mu0)
axins.plot(recorder2.data_time, np.array(recorder1.data)/mu0)
axins.set_xlim([1e-15, 5e-15])  # x limits
axins.set_ylim([-100000, 100000])  # y limits

axins.xaxis.set_major_locator(ticker.MultipleLocator(base=1e-15))

# Create a dashed rectangle around the x and y limits of the inset
mark_inset(ax, axins, loc1=2, loc2=1, fc="none", ec="0.5")

plt.show()


In [ ]:
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 75
plt.ioff()
v = np.max(test.data_yee)*0.5
anim = test.animate_field(v)
from IPython.display import HTML
HTML(anim.to_jshtml())


In [ ]:
%matplotlib inline

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 75
plt.ioff()
v = np.max(test.data_yee[200:])*1.2
anim = test.animate_field(v)
HTML(anim.to_jshtml())


In [ ]:
%matplotlib inline
plt.plot(test.recorders[0].data)


In [ ]:
%matplotlib inline
QMscheme1.expvalues('energy')



In [ ]:
QMscheme2.expvalues('energy')


In [ ]:
QMscheme1.heatmap()
